In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

class DimensionalityReducer:
    def __init__(self, method='pca', config=None):
        self.method = method.lower()
        self.config = config if config is not None else {}
        self.model = None
        self.fitted = False

    def fit(self, X, y=None):
        if self.method == 'pca':
            self._fit_pca(X)

        elif self.method == 'tsne':
            self._fit_tsne(X)

        else:
            raise ValueError(f"Method '{self.method}' not supported. Choose 'pca' or 'tsne'.")

        self.fitted = True
        return self

    def transform(self, X):
        if not self.fitted:
            raise RuntimeError("Model must be fitted before transform.")

        if self.method == 'pca':
            return self.model.transform(X)

        elif self.method == 'tsne':

            raise NotImplementedError("t-SNE does not support transform on new data. Use fit_transform instead.")

        else:
            raise ValueError(f"Method '{self.method}' not supported.")

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        if self.method == 'pca':
            return self.model.transform(X)

        elif self.method == 'tsne':
            return self.model.fit_transform(X)

        else:
            raise ValueError(f"Method '{self.method}' not supported.")

    def _fit_pca(self, X):
        n_components = self.config.get('n_components', None)
        variance_threshold = self.config.get('variance_threshold', None)
        svd_solver = self.config.get('svd_solver', 'auto')
        whiten = self.config.get('whiten', False)
        random_state = self.config.get('random_state', None)

        if variance_threshold is not None:
            if n_components is not None:
                print("Warning: Both n_components and variance_threshold provided. Using variance_threshold.")

            pca_temp = PCA(svd_solver=svd_solver, random_state=random_state)
            pca_temp.fit(X)
            explained_variance_ratio = pca_temp.explained_variance_ratio_
            cumulative = np.cumsum(explained_variance_ratio)

            n_components = np.searchsorted(cumulative, variance_threshold) + 1
            if n_components > X.shape[1]:
                n_components = X.shape[1]

            print(f"Selected {n_components} components to explain at least {variance_threshold*100:.1f}% variance.")

        elif isinstance(n_components, float) and 0 < n_components < 1:
            pca_temp = PCA(svd_solver=svd_solver, random_state=random_state)
            pca_temp.fit(X)
            explained_variance_ratio = pca_temp.explained_variance_ratio_
            cumulative = np.cumsum(explained_variance_ratio)
            n_components = np.searchsorted(cumulative, n_components) + 1
            if n_components > X.shape[1]:
                n_components = X.shape[1]

            print(f"Selected {n_components} components to explain at least {n_components*100:.1f}% variance.")

        self.model = PCA(
            n_components=n_components,
            svd_solver=svd_solver,
            whiten=whiten,
            random_state=random_state
        )
        self.model.fit(X)

    def _fit_tsne(self, X):
        n_components = self.config.get('n_components', 2)
        perplexity = self.config.get('perplexity', 30)
        early_exaggeration = self.config.get('early_exaggeration', 12.0)
        learning_rate = self.config.get('learning_rate', 200.0)
        n_iter = self.config.get('n_iter', 1000)
        n_iter_without_progress = self.config.get('n_iter_without_progress', 300)
        min_grad_norm = self.config.get('min_grad_norm', 1e-7)
        metric = self.config.get('metric', 'euclidean')
        init = self.config.get('init', 'random')
        verbose = self.config.get('verbose', 0)
        random_state = self.config.get('random_state', None)
        method = self.config.get('method', 'barnes_hut')
        angle = self.config.get('angle', 0.5)

        self.model = TSNE(
            n_components=n_components,
            perplexity=perplexity,
            early_exaggeration=early_exaggeration,
            learning_rate=learning_rate,
            n_iter=n_iter,
            n_iter_without_progress=n_iter_without_progress,
            min_grad_norm=min_grad_norm,
            metric=metric,
            init=init,
            verbose=verbose,
            random_state=random_state,
            method=method,
            angle=angle
        )

        pass

    def get_params(self):
        if self.model is not None:
            return self.model.get_params()

        else:
            return {}